Motivated by @Yi-Xuan, I will experiment the model with global pooling, instead of zero padding!

In [1]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [13]:
from pathlib import Path
drive_path = Path('/gdrive/Shareddrives/Dion-Account/2122WS/8-dl4slp/coding-project/ser/')

In [6]:
Path.ls = lambda x: list(x.iterdir())

In [14]:
drive_data_path = drive_path/'data'

In [23]:
(drive_data_path).ls()

[PosixPath('/gdrive/Shareddrives/Dion-Account/2122WS/8-dl4slp/coding-project/ser/data/v0'),
 PosixPath('/gdrive/Shareddrives/Dion-Account/2122WS/8-dl4slp/coding-project/ser/data/v1'),
 PosixPath('/gdrive/Shareddrives/Dion-Account/2122WS/8-dl4slp/coding-project/ser/data/v2.1'),
 PosixPath('/gdrive/Shareddrives/Dion-Account/2122WS/8-dl4slp/coding-project/ser/data/ser-data-v2-1.tar.gz'),
 PosixPath('/gdrive/Shareddrives/Dion-Account/2122WS/8-dl4slp/coding-project/ser/data/ser.tar.gz')]

In [24]:
!tar -xf {drive_data_path/'ser-data-v2-1.tar.gz'} -C {data_path}

@@ In summary: to do global pooling:
1. extract features from time frame using channels
2. cnn and adaptive pooling!

In [17]:
data_path = Path('/content/data'); data_path.mkdir()

In [19]:
!tar -xf {drive_data_path/'v0/ser.tar.gz'}  -C {data_path}

In [133]:
features = [torch.load(path).t() for path in (data_path/'ser/dev').ls()[:100]]

In [47]:
from torch import nn

In [196]:
features[0].shape

torch.Size([26, 105])

## TODO: unsqueeze(-1)

In [136]:
t1= features[0].unsqueeze(1); t.shape

torch.Size([26, 1, 105])

In [153]:
model = nn.Sequential(
    nn.Conv1d(1,8, kernel_size=30, stride=1),
    nn.Conv1d(8,16, kernel_size=20),
    nn.Conv1d(16,32, kernel_size=15),
    nn.Conv1d(32,64, kernel_size=10),
    nn.Conv1d(64,128, kernel_size=5),
    nn.Conv1d(128,128, kernel_size=3), #26, 128, 28
    nn.AdaptiveAvgPool2d((128, 1)), #26, 128, 1
    )

In [154]:
one_out = model(t1); one_out.shape

torch.Size([26, 128, 1])

In [147]:
x = one_out

In [156]:
one_out.shape == (26, 128, 1)

True

In [162]:
features[0].shape

torch.Size([26, 105])

In [164]:
from collections import Counter

In [175]:
dev_frame_length = sorted([(idx, len(i[1])) for idx, i in enumerate(features)], key = lambda x: x)

In [176]:
dev_frame_length[:10]

[(72, 54),
 (43, 55),
 (5, 58),
 (76, 72),
 (47, 75),
 (60, 77),
 (44, 80),
 (14, 82),
 (92, 82),
 (48, 84)]

In [177]:
features[72].shape

torch.Size([26, 54])

- let me see i can apply this to all dev set

In [ ]:
for idx, inst in enumerate(features):
    out = model(inst.unsqueeze(1))
    assert out.shape == one_out.shape, f"{idx} data has shape of {inst.shape}"


In [185]:
x = features[72].unsqueeze(1); x.shape

torch.Size([26, 1, 54])

In [180]:
model

Sequential(
  (0): Conv1d(1, 8, kernel_size=(30,), stride=(1,))
  (1): Conv1d(8, 16, kernel_size=(20,), stride=(1,))
  (2): Conv1d(16, 32, kernel_size=(15,), stride=(1,))
  (3): Conv1d(32, 64, kernel_size=(10,), stride=(1,))
  (4): Conv1d(64, 128, kernel_size=(5,), stride=(1,))
  (5): Conv1d(128, 128, kernel_size=(3,), stride=(1,))
  (6): AdaptiveAvgPool2d(output_size=(128, 1))
)

In [193]:
model = nn.Sequential(
    nn.Conv1d(1,8, kernel_size=20, stride=1),
    nn.Conv1d(8,16, kernel_size=15),
    nn.Conv1d(16,32, kernel_size=7),
    nn.Conv1d(32,64, kernel_size=5),
    nn.Conv1d(64,128, kernel_size=3),
    nn.Conv1d(128,128, kernel_size=3), #26, 128, 28
    nn.AdaptiveAvgPool2d((128, 1)), #26, 128, 1
    )

In [194]:
for i in range(len(model)):
    print(f"{i+1} layer: {model[:i+1](x).shape}")

1 layer: torch.Size([26, 8, 35])
2 layer: torch.Size([26, 16, 21])
3 layer: torch.Size([26, 32, 15])
4 layer: torch.Size([26, 64, 11])
5 layer: torch.Size([26, 128, 9])
6 layer: torch.Size([26, 128, 7])
7 layer: torch.Size([26, 128, 1])
